In [98]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## 1) Prepare URL

In [93]:
def get_url(position, location):#start):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'#&start={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    #start = start.replace(' ', '+')
    url = template.format(position, location)
    return url

### 1a) Get Clinical Informatics Jobs

In [94]:
url = get_url('Clinical Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Clinical+Informatics&l=USA


### 1b) Get Health Informatics Jobs

In [95]:
url = get_url('Health Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Health+Informatics&l=USA


## 2) Search Jobs

In [99]:
response = requests.get(url)
#print(response)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)#job_seen_beacon
cards = soup.find_all('div', 'job_seen_beacon')
#print(cards)
print(len(cards))

15


## 3) Get job post

In [7]:
def get_record(card):
    """Extract job data from a single record"""
    job_header= card.table.tbody.tr.td
    job_title = job_header.div.h2.a.span.get('title')
    job_link= 'https://www.indeed.com'+job_header.div.h2.a.get('href')

    try:
      job_company=job_header.find('span','companyName').text.strip()
    except:
      print("No company name found for job "+job_title)

    try:
      job_ratings=job_header.find('span','ratingsDisplay').text.strip()
    except:
      print("No job ratings found for job "+job_title)
      job_ratings=""

    try:
      job_location=job_header.find('div','companyLocation').text.strip()
    except:
      print("No job location found for job "+job_title)

    try:
      job_date=card.find('span','date').text.strip().replace("Posted", "").replace("EmployerActive", "").strip()
      print(job_date)
    except:
      print("No job date found for job "+job_title)

    try:
      job_snippets=card.find_all('div','attribute_snippet')
      print(len(job_snippets))
    except:
      #print("No job location found for job "+job_title)
      job_snippets=""

    if job_snippets!="":
        try:      
          job_salary=job_snippets.find('svg',{'aria-label':'Salary'})
        except:
          #print("No job salary found for job "+job_title)
          job_salary=""
        try:      
          job_type=job_snippets.find('svg',{'aria-label':'Job type'})
        except:
          #print("No job type found for job "+job_title)
          job_type=""
        try:      
          job_shift=job_snippets.find('svg',{'aria-label':'Shift'})
        except:
          #print("No job shift found for job "+job_title)
          job_shift=""
    else:
        job_salary=""
        job_type=""
        job_shift=""


    #print(job_title)
    #print(job_link)
    #print(job_company)
    #print(job_location)
    #print(job_salary)
    #print(job_type)
    #print(job_shift)
    try:      
        job_brief=card.find('div','job-snippet').ul.li.text.strip().replace('\n', ' ')
    except:
        #print("No job summary found for job "+job_title)
        job_brief=""
    
    print(job_brief)

    collect_date = datetime.today().strftime('%Y-%m-%d')
    #print(collect_date)        
    record = (job_title, job_link, job_company, job_location, job_date, collect_date, job_brief, job_ratings,job_salary, job_type, job_shift)
    return record

## 4) Get all jobs by scraping the next pages of the search

In [ ]:
records=[]
url=get_url('Health Informatics', 'USA') 
#url=get_url('Clinical Informatics', 'USA') # Uncomment and comment the above line to retrieve the Health Informatics Jobs
for i in range(67):
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')

    for card in cards:
        record = get_record(card)
        if record in records:
            continue
        else:
            records.append(record)
    try: 
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        print('search results error in iter '+str(i))
        break

In [26]:
print(len(records))

831


## 5) Save job posts in file

In [27]:
# save the job data
with open('results.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['JobTitle', 'JobUrl','Company', 'Location', 'PostDate', 'ExtractDate', 'Summary','Job_ratings', 'Salary', 'JobType','JobShift'])
    writer.writerows(records)

## 6) Getting jobs briefing: title, URL, summary, location, etc from the search result of the previous steps

In [6]:
from selenium import webdriver
import pandas as pd
jobs = pd.read_csv('results.csv')
titles = jobs['JobTitle'].tolist()
urls = jobs['JobUrl'].tolist()
company = jobs['Company'].tolist()
location= jobs['Company'].tolist()
PostDate= jobs['PostDate'].tolist()
ExtractDate= jobs['ExtractDate'].tolist()
Summary= jobs['Summary'].tolist()
Job_ratings= jobs['Job_ratings'].tolist()
Salary= jobs['Salary'].tolist()
JobType= jobs['JobType'].tolist()
JobShift= jobs['JobShift'].tolist()

In [17]:
# Installing the web driver for web browser
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - There is no [mac64] chromedriver for browser 101.0.4951 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_1115/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [100]:
import os
# The below is the path of the web browser driver on the machine used for this research.
# To reuse, change the below file location to the location of the web browser driver on your local machine
os.path.exists('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41')

True

## 7) Getting jobs details by opening each job URL

In [ ]:
import sys
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
count=0
jobsdesc=[]
for l in urls:
    url = l
    # initiating the webdriver. Parameter includes the path of the webdriver.
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    try:
        driver.get(url) 

        # this is just to ensure that the page is loaded
        time.sleep(5) 

        html = driver.page_source
        # this renders the JS code and stores all
        # of the information in static HTML code.

        # Now, we could simply apply bs4 to html variable
        soup = BeautifulSoup(html, "html.parser")

        try:
            salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
            Salary[count]=salary
        except:
            print("No salary found for job ")
        jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
        #print(jobDescription)
        jobsdesc.append(jobDescription)
    except:
        print("Description of job number "+str(count)+" was not retrieved")
        jobsdesc.append("")
    #print(driver.find_element(By.XPATH, '//*[@id="jobDetailsSection"]/div[2]/div[2]')).text
    #document.querySelector("#jobDetailsSection > div.jobsearch-JobDescriptionSection-sectionItem > div:nth-child(2)")
    #all_divs = soup.find('div', {'id' : 'nameSearch'})
    #job_profiles = all_divs.find_all('a')

    # printing top ten job profiles
    #count = 0
    #for job_profile in job_profiles :
        #print(job_profile.text)
        #count = count + 1
        #if(count == 10) :
            #break
    count=count+1
    
driver.close() # closing the webdriver

In [33]:
print(len(jobsdesc))

571


In [8]:
## Checking for error loading the job descriptions pages detail
diff=len(urls)-count
print(diff)
print(count)

0
831


## 8) load remaining jobs that did not load in previous point due to error loading

In [ ]:
for i in range(count, len(jobsdesc)):
    url=urls[u]
    print(url)
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    driver.get(url) 

    # this is just to ensure that the page is loaded
    time.sleep(5) 

    html = driver.page_source
    # this renders the JS code and stores all
    # of the information in static HTML code.

    # Now, we could simply apply bs4 to html variable
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
        Salary[count]=salary
    except:
        print("No salary found for job ")
        
        
    jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
    #print(jobDescription)
    jobsdesc.append(jobDescription)
    
    #print(driver.find_element(By.XPATH, '//*[@id="jobDetailsSection"]/div[2]/div[2]')).text
    #document.querySelector("#jobDetailsSection > div.jobsearch-JobDescriptionSection-sectionItem > div:nth-child(2)")
    #all_divs = soup.find('div', {'id' : 'nameSearch'})
    #job_profiles = all_divs.find_all('a')

    # printing top ten job profiles
    #count = 0
    #for job_profile in job_profiles :
        #print(job_profile.text)
        #count = count + 1
        #if(count == 10) :
            #break
    count=count+1

## 9) save jobs with job description into csv

In [29]:
df_updated = pd.DataFrame({'Job_titles':titles,'Job_URLS':urls,'Company':company,'location':location,'PostDate':PostDate,'ExtractDate':ExtractDate,'Summary':Summary,'Job_ratings':Job_ratings,'Job_description':jobsdesc,'Salary':Salary,'JobType':JobType,'JobShift':JobShift})
df_updated.to_csv('JobsDetails.csv')

## 10) Analyze jobs

### 10A) Clean the job description

In [25]:
import pandas as pd
df_updated = pd.read_csv(r'JobDetails.csv')

In [26]:
from html.parser import HTMLParser
import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def cleanURLEmail(strip):
    # extract emails
    match = re.findall(r'[\w\.-]+@[\w\.-]+', strip)
    temp = strip
    for m in match:
        temp = strip.replace(m, '', 100)
        strip = temp
    match3 = re.findall(r'^https?:\/\/.*[\r\n]*', strip)
    for m in match3:
        temp = strip.replace(m, '', 100)
        strip = temp
    return strip


    #Remove Stop words, punctuation, and extract lemmatization, and stemming.
def clean(doc,stop,exclude):
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop])
    no_punc = map(lambda x: re.sub("[^a-z]", " ", x), stop_free)
    punc_free = "".join(ch for ch in no_punc if ch not in exclude)
    normalized=""
    stemmed = ""
    for word in punc_free.split():
        try:
            lWord=lemma.lemmatize(word)
        except:
            continue
        normalized=normalized+lWord+" "
    stop_free2 = " ".join([i for i in normalized.lower().split() if i not in stop])
    return stop_free2

In [27]:
from selenium import webdriver
import time
import csv
import pandas as pd
#import simplejson
from nltk import ngrams
import datetime
import re
import string
import nltk
import requests
from nltk.corpus import stopwords
import unicodedata



driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
statesBSC={}
statesMaster={}
statesCertifications={}
states={}
for k, v in us_state_abbrev.items():
    statesBSC[v]=0
    statesMaster[v] = 0
    statesCertifications[v] = 0
    states[v]=0


def cleanJobDescription(description):
    doc_complete=[]
    doc_clean=[]
    tList=[]
    exclude = set(string.punctuation)
    stop = set(stopwords.words('english'))
    lines = open("stop3").read().splitlines()
    for word in lines:
        #print(word)
        stop.add(word)
    myStop = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all",
                 "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst",
                 "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway",
                 "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes",
                 "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides",
                 "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co",
                 "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due",
                 "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc",
                 "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen",
                 "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found",
                 "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have",
                 "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
                 "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed",
                 "interest", "into", "itself", "keep", "last", "latter", "latterly", "least",
                 "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more",
                 "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither",
                 "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                 "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other",
                 "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
                 "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious",
                 "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                 "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system",
                 "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there",
                 "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin",
                 "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to",
                 "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until",
                 "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
                 "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
                 "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
                 "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself",
                 "yourselves", "the", 'http', 'https', '://', 'www', 'com', '8800', '...', '....','//','/','..', 'yep', '.).', '](#', '.:).',
                '++..', 'github', 'etc', 'also', 'org', 'gee', 'let', 'know', 'ever',
                'vcntr', 'falseamount', 'isig']
    for word in myStop:
        stop.add(word)
        
    #strip HTML tags from tweet
    temp=""
    #strip = strip_tags(description)
    #strip1 = cleanURLEmail(description)
    strip1 = str(description).replace(".","")
    strip2 = clean(strip1,stop,'')
    return(strip2)



/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_812/3880222035.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver')


In [34]:
print(cleanJobDescription("MD"))

md


In [28]:
df_updated['cleanDesc']= df_updated.apply(lambda x : cleanJobDescription(x['Job_description']),axis=1)

In [17]:
print(df_updated['cleanDesc'])

0      galen award winning healthcare technical profe...
1      deep mile network llc deep mile network llc pr...
2      remote usa time cmo people passionate solving ...
3      centura health largest health ecosystem colora...
4      national coordination center ncc powered edera...
                             ...                        
826    location loc broomall care network broomall re...
827    type requisition regular clearance level posse...
828    overview seeking human resource support specia...
829    emerald health service hiring travel ray techn...
830    job description real world data engineer ii de...
Name: cleanDesc, Length: 831, dtype: object


### 10b) Get skills, certificates, training, and degrees

In [103]:
# Funcation to flag the job if it required or prefer a professional certification or a university degree
def checkDegrees(description,degreeCheck):
    result=0
    graduateKeywords=['mba','msc', 'md', 'master', 'phd', 'doctorate']
    bachelorKeywords = ['college', 'bachelor', 'bsc']
    degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
    if degreeCheck in str(description):
        result= 1
    elif degreeCheck == 'bachelor':
        for word in bachelorKeywords:
            if word in str(description):
                result = 1
                break
    elif degreeCheck == 'graduate':
        for word in graduateKeywords:
            if word in str(description):
                result = 1
                break
    elif degreeCheck == 'certifications':
        if "ertified" in str(description) or "ertificat" in str(description):
                result = 1
    elif degreeCheck == 'otherDegree':
        count=0
        if 'degree' in str(description):
            for word in degrees:
                if word in str(description):
                    result = 0
                    break
                count=count+1
                if count>8: 
                    result = 1
    return result

In [104]:
# Function that extracts the certifications, skills, programming languages, data visualization tools required 
# or prefered in the job description
# check to exclude certification if has any of the following degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
def getText(description, search):
    count = 0
    degree=0
    # For professional certifications exclude, any academic certification 
    degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
    try:
        TexttotSet=set()
        words=description.lower().split()
        count=0
        cert=0
        for w in words:
            temp=""
            if (search=="certifications" and ("ertified" in w or "ertificat" in w)) or (search=="skills" and "skill" in w) or (search=="trainings" and "train" in w)or (search=="degrees" and "degree" in w) or (search=="license" and "license" in w) or (search=="visualization" and "visualization" in w) or (search=="programming" and "progamming" in w):
                degree=0
                counter=4
                i=count-1
                tempQueue=[]
                while(i>-1 and counter>0):
                    if words[i] in degrees:
                        degree=1
                        break
                    tempQueue.append(words[i])
                    counter-=1
                    i-=1
                if degree==0:
                    while(len(tempQueue)>0):
                        temp=tempQueue.pop(0)+" "+temp
                counter=1
                while (degree==0 and counter<5 and counter+count<len(words)):
                    if words[count+counter] in degrees:
                        degree=1
                        break
                    temp=temp+" "+words[count+counter]
                    counter+=1
                if degree==0:
                    TexttotSet.add(temp)
            count+=1
        
        # Extract the certification or skill name that could consists of one word, two words, three words...upto six words
        s=""
        for t in TexttotSet:
            Text=str(t)
            #Bigrams
            bigramsText=""
            n = 2
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]
                bigramsText=bigramsText+" "+temp

            #trigrams
            trigramsText = ""
            n = 3
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]
                trigramsText = trigramsText + " " + temp

            fourgramsText = ""
            n = 4
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]
                fourgramsText = fourgramsText + " " + temp

            fivegramsText=""
            n = 5
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]
                fivegramsText = fivegramsText + " " + temp

            sixgramsText=""
            n = 6
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]+"_"+grams[5]
                sixgramsText = sixgramsText + " " + temp

            s=s+Text+" "+bigramsText+" "+trigramsText+" "+fourgramsText+" "+fivegramsText+" "+sixgramsText+" "
            return s.strip()
    except Exception as e:
        print(e)
        

In [85]:
df_updated['certifications']= df_updated.apply(lambda x : getText(x['cleanDesc'],'certifications'),axis=1)
df_updated['skills']= df_updated.apply(lambda x : getText(x['cleanDesc'],'skills'),axis=1)
df_updated['trainings']= df_updated.apply(lambda x : getText(x['cleanDesc'],'trainings'),axis=1)
df_updated['degrees']= df_updated.apply(lambda x : getText(x['cleanDesc'],'degrees'),axis=1)
df_updated['license']= df_updated.apply(lambda x : getText(x['cleanDesc'],'license'),axis=1)
df_updated['programming']= df_updated.apply(lambda x : getText(x['cleanDesc'],'programming'),axis=1)
df_updated['visualization']= df_updated.apply(lambda x : getText(x['cleanDesc'],'visualization'),axis=1)

In [44]:
df_updated['bachelor']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'bachelor'),axis=1)
df_updated['graduate']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'graduate'),axis=1)
df_updated['otherDegree']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'otherDegree'),axis=1)

In [106]:
def checkCollege(bachelor,graduate):
    if bachelor ==1 or graduate==1:
        return 1
    else:
        return 0
df_updated['college']=df_updated.apply(lambda x : checkCollege(x['bachelor'],x['graduate']),axis=1)

In [107]:
df_updated[df_updated['college']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
32,32,32,Health Informatics and Reporting Analyst,https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Working knowledge of the Massachusetts health ...,NaN,...,0,provide documentation vaccine medically objec...,public administration economics similar subst...,1,1,MA,0,None,None,excel powerpoint familiarity data business in...
37,37,37,Informatics Analyst,https://www.indeed.com/rc/clk?jk=d4b54cd7730b1...,The University of Chicago Medicine,"Chicago, IL 60637 (Hyde Park area)",3 days ago,2022-05-23,Knowledge of the US health care system preferred.,3.8,...,1,None,None,0,1,IL,0,None,None,None
39,39,39,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=455c2d5b2baf1...,University of California San Francisco,Remote,17 days ago,2022-05-23,Broadly encompassing or in-depth knowledge of ...,4.1,...,1,area equivalent combination experience year e...,None,1,1,0,0,None,None,None
44,44,44,"Associate Manager, Health Informatics and Repo...",https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Monitor emerging topics and news in health pol...,NaN,...,0,procedure provide guidance mentoring hir team...,None,1,1,MA,0,None,None,excel powerpoint familiarity data business in...
49,49,49,Adjunct Faculty – Healthcare Informatics,https://www.indeed.com/rc/clk?jk=e8894c65cd061...,MCPHS University,Remote,30+ days ago,2022-05-23,Minimum of a master’s degree in health science...,3.9,...,0,None,None,1,1,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,801,801,SYSTEMS PROJECT CONSULTANT,https://www.indeed.com/rc/clk?jk=1cb57a27f8493...,The State of Florida,"Tallahassee, FL 32308",30+ days ago,2022-05-23,The selected candidate will assist in detectin...,3.5,...,0,data report mpi multitude opportunity mpi emp...,None,0,1,FL,0,None,None,python creation interactive data tool tableau...
805,805,805,Data Analyst,https://www.indeed.com/company/NYSTEC/jobs/Dat...,NYSTEC,"Albany, NY 12207+1 location",20 days ago,2022-05-23,Use of health informatics or Medicaid/Medicare...,4.0,...,0,equivalent combination advanced education exp...,None,0,1,NY,0,None,None,None
811,811,811,Clinical Informatics Specialist 3 - * Days - 4...,https://www.indeed.com/company/University-of-M...,Metro Health - University of Michigan Health,"Wyoming, MI",Hiring ongoing,2022-05-23,Certification in nursing informatics preferred.,3.7,...,0,education program responsible accuracy conten...,None,0,1,MI,848,None,None,None
814,814,814,Public Health Informaticist,https://www.indeed.com/rc/clk?jk=be30904d64b13...,Weber County,"Ogden, UT 84401",30+ days ago,2022-05-23,Ability to integrate and incorporate informati...,2.7,...,0,None,None,0,1,UT,0,None,None,None


In [108]:
df_updated[df_updated['graduate']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
32,32,32,Health Informatics and Reporting Analyst,https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Working knowledge of the Massachusetts health ...,NaN,...,0,provide documentation vaccine medically objec...,public administration economics similar subst...,1,1,MA,0,None,None,excel powerpoint familiarity data business in...
37,37,37,Informatics Analyst,https://www.indeed.com/rc/clk?jk=d4b54cd7730b1...,The University of Chicago Medicine,"Chicago, IL 60637 (Hyde Park area)",3 days ago,2022-05-23,Knowledge of the US health care system preferred.,3.8,...,1,None,None,0,1,IL,0,None,None,None
39,39,39,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=455c2d5b2baf1...,University of California San Francisco,Remote,17 days ago,2022-05-23,Broadly encompassing or in-depth knowledge of ...,4.1,...,1,area equivalent combination experience year e...,None,1,1,0,0,None,None,None
49,49,49,Adjunct Faculty – Healthcare Informatics,https://www.indeed.com/rc/clk?jk=e8894c65cd061...,MCPHS University,Remote,30+ days ago,2022-05-23,Minimum of a master’s degree in health science...,3.9,...,0,None,None,1,1,0,0,None,None,None
53,53,53,Health Informatics Part Time Lecturer,https://www.indeed.com/rc/clk?jk=2328b797f7e07...,Northeastern University,"Boston, MA+1 location",30+ days ago,2022-05-23,Teach and develop on-line or on-campus courses...,4.2,...,1,None,None,0,1,MA,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,788,788,Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=3a02f7e4353ba...,Central Peninsula Hospital,"Soldotna, AK 99669",26 days ago,2022-05-23,Advanced Degree in Healthcare Informatics rela...,3.7,...,0,software support maintenance troubleshooting ...,industry terminology preferred advanced healt...,0,1,AK,987,None,None,None
799,799,799,Health and Safety Fall Co-Op Positions,https://www.indeed.com/rc/clk?jk=e296525257a4e...,Cummins Inc.,"Columbus, IN",30+ days ago,2022-05-23,Fundamentals of Health and Safety - Comprehens...,3.8,...,0,None,student pursuing undergraduate graduate minim...,0,1,IN,0,None,None,None
801,801,801,SYSTEMS PROJECT CONSULTANT,https://www.indeed.com/rc/clk?jk=1cb57a27f8493...,The State of Florida,"Tallahassee, FL 32308",30+ days ago,2022-05-23,The selected candidate will assist in detectin...,3.5,...,0,data report mpi multitude opportunity mpi emp...,None,0,1,FL,0,None,None,python creation interactive data tool tableau...
814,814,814,Public Health Informaticist,https://www.indeed.com/rc/clk?jk=be30904d64b13...,Weber County,"Ogden, UT 84401",30+ days ago,2022-05-23,Ability to integrate and incorporate informati...,2.7,...,0,None,None,0,1,UT,0,None,None,None


In [105]:
# Flag the job if a certification is requested
def checkCertificationLenght(certification):
    if certification is None:
        return 0
    return len(certification)
df_updated['certificationFlag']=df_updated.apply(lambda x : checkCertificationLenght(x['certifications']),axis=1)

In [88]:
df_updated['certificationFlag']

0         0
1         0
2       798
3         0
4      1012
       ... 
826     913
827     638
828       0
829     785
830       0
Name: certificationFlag, Length: 831, dtype: int64

In [90]:
df_updated[df_updated['certificationFlag']>0].to_csv('certifications.csv')

# The reason why 40% of the jobs listed prefering or requiring certification while 
# the jobs with specifically mentioned certifications are almost 10% to 15% is that
# most of the jobs either mentioned that certification is required or prefered without specifying like:
# "Certification relevant to training," "Industry designation certifications," "Industry-focused Certifications,"
# or "Speciality certification is required", etc. Also, some jobs mentioned "Certification support" as part 
# of the professional development benefits in the job to attract candidates focusing on continous improvement
 

In [27]:
# Checking the location of the jobs by state

# Checking jobs that have the flexibility of working remotely
def checkRemote(location):
    if 'remote' in location.lower():
        return 1
    else:
        return 0
df_updated['remote']=df_updated.apply(lambda x : checkRemote(x['location']),axis=1)

In [28]:
statesAbbrev=list(us_state_abbrev.values())
statesAb=[]
for st in statesAbbrev:
    statesAb.append(" "+st)
states=list(us_state_abbrev.keys())

In [29]:
def checkState(location):
    for st in statesAb:
        if st in location:
            return st.strip()
    for st in states:
        if st in location:
            return us_state_abbrev[st]
    return '0'
df_updated['states']=df_updated.apply(lambda x : checkState(x['location']),axis=1)

In [30]:
df_updated['states'].value_counts().to_csv('States.csv')

### N.B. 1/4 of the jobs have a remote working flexibility

In [31]:
df_updated[df_updated['remote']==1]

,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
0,0,0,Clinical Informatics Specialist - Contractor Role,https://www.indeed.com/company/Galen-Healthcar...,Galen HealthCare Solutions,Remote,24 days ago,2022-05-23,Serves as a resource person for daily operatio...,2.8,...,1,program support end user training assist appl...,understanding workflow critical bachelor degr...,1,0,0,0,,,
1,1,1,Electronic Health Record (EHR) Adoption/Optimi...,https://www.indeed.com/company/DEEPMILE/jobs/E...,DEEPMILE,Remote,3 days ago,2022-05-23,Knowledge of various specialized areas of heal...,NaN,...,1,network llc premier provider training technol...,minimum qualification bachelor level degree y...,1,0,0,0,,,
2,2,2,Clinical Informatics,https://www.indeed.com/rc/clk?jk=e1a9ee41cd34a...,Innovacer,Remote,12 days ago,2022-05-23,We are looking for a Clinical Informatics who ...,NaN,...,1,,practice building set master degree clinical ...,1,0,0,1,,,
4,4,4,Clinical Informatics Consultant (Generalist),https://www.indeed.com/company/Edera-L3C/jobs/...,Edera L3C,Remote,5 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,1,implementation workflow development testing t...,capability education experience year degree h...,1,0,0,1,,,
5,5,5,Clinical Informatics Epic Support Specialist (...,https://www.indeed.com/rc/clk?jk=8013a22efa6d1...,Providence,"Remote in Beaverton, OR 97075+21 locations",2 days ago,2022-05-23,Ensures that workflow and training knowledge d...,3.9,...,0,electronic health record ehr training compete...,qualification position include bachelor degre...,1,0,OR,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,803,803,Financial Analyst,https://www.indeed.com/rc/clk?jk=1c7e6da17c92e...,Emory University,"Temporarily Remote in Decatur, GA 30030",30+ days ago,2022-05-23,Analyzes historical accounting data and other ...,4.0,...,1,equivalent combination experience education t...,analysis minimum qualification bachelor degre...,1,0,GA,0,,,
807,807,807,Talent Development Manager,https://www.indeed.com/company/Chartis-Group/j...,Chartis Group,Remote,30+ days ago,2022-05-23,"Reporting to the VP of Talent Development, the...",NaN,...,1,,strategic tactical responsibility bachelor de...,1,0,0,0,,,
809,809,809,Business Intelligence Developer,https://www.indeed.com/company/Global-IT-Resou...,Global IT Resources,+1 locationRemote,3 days ago,2022-05-23,The incumbent will provide analytical support ...,3.7,...,0,application understand policy procedure const...,,1,0,0,0,,,data source developing report visualization a...
818,818,818,Coding Quality Auditor - Remote,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Houston Methodist,"Remote in Houston, TX+16 locations",18 days ago,2022-05-23,"At Houston Methodist, the Coding Quality Audit...",4.2,...,1,diagnosis procedure coding documentation trai...,arises education requirement associate degree...,1,0,TX,1,coder transition program certification licens...,,


### Checking job flexibility with college degree requirement (no correlation found)

In [81]:
df_updated['remote'].corr(df_updated['college'])

-0.07909216195943626

### Checking the skills, certifications, data visualization tools in highest demand

In [40]:
def getHighDemandJobDescription(df,demand):
    def preprocess(text):
        tokens = []
        tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-z]+")
        for token in tokenizer.tokenize(text):
            tokens.append(token)
        return tokens
    if demand=='skills':
        List=df['skills'].tolist()
    elif demand=='certifications':
        List=df['certifications'].tolist()
    elif demand=='trainings':
        List=df['trainings'].tolist()
    elif demand=='license':
        List=df['license'].tolist()
    elif demand=='programming':
        List=df['programming'].tolist()
    elif demand=='visualization':
        List=df['visualization'].tolist()
    else:
        List=df['degrees'].tolist()
    processed = list(map(preprocess, List))
    fdist = nltk.FreqDist([token for doc in processed for token in doc])
    print(fdist.tabulate(1000))

In [33]:
getHighDemandJobDescription(df_updated,'skills')

          skill   communication         ability          strong       excellent         written       knowledge      experience   interpersonal      management          verbal        required         problem  organizational         solving      analytical            work            oral         project    demonstrated            data       preferred        computer       technical            team       including      leadership        advanced       effective        thinking            time        customer        clinical     effectively         service    presentation          health        critical     information       microsoft         skilled         working          office     requirement        analysis     environment             job       attention           level        software         complex   qualification          degree  responsibility     communicate            good          highly       education           posse    organization         support           excel    demon

In [35]:
import numpy as np
df=df_updated
df['visualization'].replace('', np.nan).dropna(inplace=True)
print(df)

     Unnamed: 0  Unnamed: 0.1  \
0             0             0   
1             1             1   
2             2             2   
3             3             3   
4             4             4   
..          ...           ...   
826         826           826   
827         827           827   
828         828           828   
829         829           829   
830         830           830   

                                            Job_titles  \
0    Clinical Informatics Specialist - Contractor Role   
1    Electronic Health Record (EHR) Adoption/Optimi...   
2                                 Clinical Informatics   
3                       Informatics & Training Analyst   
4         Clinical Informatics Consultant (Generalist)   
..                                                 ...   
826     RN, Registered Nurse (Primary Care - Broomall)   
827               Senior Data Engineer - TS/SCI w/Poly   
828                         Product Support Specialist   
829             Travel 

In [36]:
getHighDemandJobDescription(df,'visualization')

  visualization            data            tool      experience        analysis         tableau       reporting          report       analytics        software         support        business      management           skill          design       dashboard    intelligence             sql          python     information       knowledge         perform           build       technique          health     statistical        database         insight       excellent        clinical          source      analytical         produce     requirement    presentation          create         variety         linkage            open         written        analytic         package         working         product      healthcare        develops       including      developing       effective        internal         project          result     communicate         process              sa              bi     application            work        customer     performance        research            shda         

In [38]:
maskSkill1 = df_updated['cleanDesc'].str.contains(r'analysis', na=True)
df_updated[maskSkill1]

# The number of jobs requesting each skill:
# Analytics and Problem Sovling 623
# Communication skills 337
# Oral Communication 238
# Project Management 199
# Statistics 198
# Interpersonal skills 172
# Critical thinking 67

# The top mentioned tools
# Excel 514
# SQL 150



,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
0,0,0,Clinical Informatics Specialist - Contractor Role,https://www.indeed.com/company/Galen-Healthcar...,Galen HealthCare Solutions,Remote,24 days ago,2022-05-23,Serves as a resource person for daily operatio...,2.8,...,1,program support end user training assist appl...,understanding workflow critical bachelor degr...,1,0,0,0,,,
1,1,1,Electronic Health Record (EHR) Adoption/Optimi...,https://www.indeed.com/company/DEEPMILE/jobs/E...,DEEPMILE,Remote,3 days ago,2022-05-23,Knowledge of various specialized areas of heal...,NaN,...,1,network llc premier provider training technol...,minimum qualification bachelor level degree y...,1,0,0,0,,,
4,4,4,Clinical Informatics Consultant (Generalist),https://www.indeed.com/company/Edera-L3C/jobs/...,Edera L3C,Remote,5 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,1,implementation workflow development testing t...,capability education experience year degree h...,1,0,0,1,,,
7,7,7,Data Informatics Analyst 2,https://www.indeed.com/rc/clk?jk=80c0208dff145...,WEX Inc.,Remote,20 days ago,2022-05-23,"Benefits include health, dental and vision ins...",3.3,...,0,data collaborate timeline responsible trainin...,,1,0,0,0,,,
9,9,9,Clinical Informatics Consultant (Generalist),https://www.indeed.com/rc/clk?jk=1b348001de7b3...,National Coordination Center,"Remote in Washington, DC",4 days ago,2022-05-23,The Clinical Informatics Consultant works dire...,NaN,...,1,implementation workflow development testing t...,capability education experience year degree h...,1,0,WA,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,820,820,Epidemiology Program Manager (Masters),https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,PMHCC Inc.,"Philadelphia, PA+2 locations",30+ days ago,2022-05-23,"Epidemiology, statistics, or informatics: 7 ye...",3.8,...,0,covid response philadelphia coordinate traini...,experience education requirement master degre...,0,1,PA,0,,,stream dcc produce report visualization suppl...
824,824,824,"Data Engineer, Insurance , Full time permanent",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Villanti and Company,","San Diego, CA",6 days ago,2022-05-23,Well established and growing Insurance Company...,NaN,...,0,,quicksight tableau powerbi education degree c...,0,0,CA,0,,,warehouse design experience data visualizatio...
825,825,825,Physician Informaticist - Inpatient Focus,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Centura Health,"Centennial, CO 80112+1 location",30+ days ago,2022-05-23,Explores the use of medical informatics system...,3.8,...,0,management process oversees documentation tra...,,0,0,CO,1,ehr preferred current unrestricted license pr...,,
827,827,827,Senior Data Engineer - TS/SCI w/Poly,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,General Dynamics,"Remote in McLean, VA",30+ days ago,2022-05-23,"Bachelor’s degree in information systems, info...",3.8,...,1,,code required experience bachelor degree info...,1,0,VA,1,,,


In [58]:
getHighDemandJobDescription(df_updated,'certifications')

   certification        preferred         required          license       experience        certified             epic      informatics        licensure        education     professional     registration         clinical             year      certificate           degree          nursing           health      information      requirement    qualification            skill      application          related       management       healthcare            board             work        knowledge          current       registered          program         bachelor            month            field         training          minimum        specialty       technology          medical             hire       equivalent             data           coding         relevant            nurse           master           obtain               cc         national            state            valid          ability               rn          project         assigned         practice       specialist         positi

In [20]:
# checking in how many jobs certain certifications was listed
mask1 = df_updated['cleanDesc'].str.contains(r'ecfmg', na=True)
df_new=df_updated[mask1]

In [107]:
# checking in how many jobs certain certifications was listed
mask3 = df_updated['cleanDesc'].str.contains(r'cpc', na=True)
df_updated[mask3]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,...,bachelor,graduate,skills,otherDegree,trainings,degrees,remote,college,states,certificationFlag
10,10,Health Informatics and Clinical Terminology Sp...,https://www.indeed.com/rc/clk?jk=56eb316ea1a9f...,ClosedLoop,"Remote in Austin, TX 78704",3 days ago,2022-05-23,"Bachelor's degree in related health field, e.g...",4.3,About ClosedLoop:\n\nClosedLoop.ai is healthca...,...,0,0,strong written oral communication skill inclu...,1,,terminology ontology qualification bachelor d...,1,0,TX,0
71,71,Clinical Data Standards Lead - Center for Heal...,https://www.indeed.com/rc/clk?jk=757b599ee3e48...,RTI International,"Remote in Durham, NC 27709",12 days ago,2022-05-23,The candidate must have experience with health...,3.6,RTI International's Translational Health Scien...,...,0,0,,1,,required minimum qualification bachelor degre...,1,0,NC,1
79,79,CHESS: Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=ec9aca1a1cb77...,MedCost LLC,Remote,4 days ago,2022-05-23,Familiar with clinical and administrative work...,3.1,Clinical Informatics Analyst\n\nPosition Summa...,...,0,0,excellent written verbal communication skill ...,1,data solution department process training man...,position qualification required bachelor degr...,1,0,0,0
115,115,Sr Clinical Informatics Consult - Telecommute,https://www.indeed.com/rc/clk?jk=24223b945a8c2...,Optum,"Remote in Draper, UT 84020",30+ days ago,2022-05-23,2+ years’ experience in leading clinical opera...,3.4,\n\n\nCombine two of the fastest-growing field...,...,0,1,problem ability apply process skill knowledge...,1,payer cost saving staff training participate ...,interested required qualification undergradua...,1,1,UT,1
201,201,Clinical Consultant (Nurse or Physician Assist...,https://www.indeed.com/rc/clk?jk=86b9b8936a8be...,Milliman,"Brookfield, WI 53005",30+ days ago,2022-05-23,Clinical informatics and/or electronic health ...,3.6,\n\n\n\n\n Job Details\n \n\n\nDescript...,...,0,0,excellent verbal written communication skill ...,0,nursing physician assistant experience traini...,,0,0,WI,0
242,242,Sr Health Data Analyst: remote,https://www.indeed.com/rc/clk?jk=a4dd5f9bc56db...,MEDSTAR HEALTH,"Remote in Rosedale, MD 21237",30+ days ago,2022-05-23,The SHDA’s remaining majority effort is applie...,3.6,\n\n\n\n\n\nJob Summary\nThe Senior Health Dat...,...,0,1,analyst shda draw specialized skillsets healt...,0,beneficiary minimum qualification education t...,qualification education training bachelor deg...,1,1,MD,1
350,350,Quality Improvement - Director,https://www.indeed.com/rc/clk?jk=48963d1a27368...,Quality Improvement & Informatics - The Corval...,"Corvallis, OR 97330",4 days ago,2022-05-23,Or minimum ten (10) years in health care (proj...,NaN,The Corvallis Clinic is open to filling this r...,...,0,0,contract program preferred knowledge skill st...,1,,licensure experience required bachelor degree...,0,0,OR,1
373,373,Remote Lead,https://www.indeed.com/rc/clk?jk=8cda48609d3cd...,Northwestern Medicine,"Remote in Chicago, IL",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,"\n\n At Northwestern Medicine, every patient ...",...,0,1,,0,issue reported end user training document tip...,meditech clinical preferred bachelor degree a...,1,1,IL,1
394,394,Remote Coding Specialist II,https://www.indeed.com/rc/clk?jk=3126d25924eb5...,Northwestern Medicine,"Remote in Chicago, IL 60611+3 locations",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,"\n\nAt Northwestern Medicine, every patient in...",...,0,0,,1,reflects patient service position train physi...,test preferred bachelor associate degree heal...,1,0,IL,1
400,400,"Vice President, Clinical Informatics",https://www.indeed.com/rc/clk?jk=54509c2cfa08a...,Cohere Health,"Remote in Boston, MA",30+ days ago,2022-05-23,5+ years of experience managing and implementi...,3.6,Company Overview:\n\nCohere 

In [111]:
# checking in how many jobs certain certifications was listed
mask4 = df_updated['cleanDesc'].str.contains(r'pmp', na=True)
df_updated[mask4]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,...,bachelor,graduate,skills,otherDegree,trainings,degrees,remote,college,states,certificationFlag
102,102,MHS Health Informatics - Senior Consultant,https://www.indeed.com/rc/clk?jk=333c280f648d1...,Guidehouse,"Tysons Corner, VA",5 days ago,2022-05-23,Provide health informatics subject matter expe...,3.3,\n\n Overview: \n \n Guidehouse is a lead...,...,0,0,excellent written verbal communication skill ...,0,,requirement qualification required bachelor d...,0,0,VA,1
124,124,Medical Economics and Informatics Analyst,https://www.indeed.com/company/HealthHelp---A-...,HealthHelp,Remote,Employer30+ days ago,2022-05-23,"Knowledge of health system functions, terminol...",NaN,\nSUMMARY\nResponsible for conducting analyses...,...,0,0,visualization tool intermediate advanced skil...,0,,needed qualification requirement bachelor deg...,1,0,0,0
311,311,Clinical Informatics Manager,https://www.indeed.com/rc/clk?jk=ba53801b1324c...,Vail Health Hospital,"Vail, CO",22 days ago,2022-05-23,Translates impact of public-policy initiatives...,NaN,\n\nPay Range: Min: $45.62 Mid: $65.17. Pay is...,...,0,1,construed exhaustive list responsibility skil...,0,regulatory data implementation education trai...,improvement concept education bachelor degree...,0,1,CO,1
386,386,Healthcare Informatics – Data Scientist,https://www.indeed.com/company/Carson-Solution...,Carson Solutions LLC,"Washington, DC 20001 (Judiciary Square area)",14 days ago,2022-05-23,Incumbent performs health informatics activiti...,NaN,\nThe Office of Information Technology is look...,...,0,1,activity requires specialized knowledge skill...,0,development implementation evaluation mainten...,cognosh phd preferred master degree economics...,0,1,WA,1
433,433,Registered Nurse - PACU/ ECT at Penn Medicine ...,https://www.indeed.com/rc/clk?jk=a09bc4d7b024a...,Penn Medicine -Prod,"Princeton, NJ 08540+1 location",3 days ago,2022-05-23,Links initiatives or projects to Penn Medicine...,NaN,\n\nDescription Penn Medicine is dedicated to ...,...,0,0,nurse colleague posse knowledge skill attitud...,0,,art science required bachelor degree nursing ...,0,0,NJ,1
443,443,Nurse Residency Program - Penn Medicine Prince...,https://www.indeed.com/rc/clk?jk=9decc46c17ac8...,Penn Medicine -Prod,"Plainsboro, NJ 08536",5 days ago,2022-05-23,The Clinical Nurse 1 New to Practice/Apprentic...,NaN,Description\nPenn Medicine is dedicated to our...,...,0,1,emphasize develop clinical leadership skill a...,0,consist mandatory seminar classroom training ...,art science required bachelor degree nursing ...,0,1,NJ,0
498,498,Technical Project Manager,https://www.indeed.com/company/Nephrology-Asso...,Nephrology Associates of Northern Illinois and...,"Oak Brook, IL 60523",7 days ago,2022-05-23,"Benefits including health, dental, 401k, PTO, ...",2.5,\nAbout NANI: \nNANI is the nation’s largest N...,...,0,0,analytical thinking problem solving skill exc...,1,personnel deployment establishes implement tr...,idea project requirement bachelor degree comp...,0,0,IL,1
550,550,CareAllies Operations Senior Manager - Work fr...,https://www.indeed.com/rc/clk?jk=52741193e2930...,Cigna,"Remote in Hartford, CT 06152",30+ days ago,2022-05-23,Foundational understanding of health care info...,3.7,"\n Remote, Work from home \n \n The Operations...",...,0,0,rulemaking preferred office advanced skill wo...,1,,strategic initiative qualification bachelor d...,1,0,CT,0
576,576,Director of Business Informatics,https://www.indeed.com/rc/clk?jk=cc8f871e12151...,Peak Vista Community Health Centers,"Colorado Springs, CO",13 days ago,2022-05-23,The DBI will oversee the implementation and de...,3.1,\n Peak Vista Community Health Centers is a no...,...,0,0,requirement listed representative knowledge s...,1,care includes developing deploying training s...,education experience education bachelor degre...,0,0,CO,1
604,604,"Applications Development Director, Em

In [112]:
# checking in how many jobs certain certifications was listed
mask5 = df_updated['cleanDesc'].str.contains(r'rhit', na=True)
df_updated[mask5]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,...,bachelor,graduate,skills,otherDegree,trainings,degrees,remote,college,states,certificationFlag
115,115,Sr Clinical Informatics Consult - Telecommute,https://www.indeed.com/rc/clk?jk=24223b945a8c2...,Optum,"Remote in Draper, UT 84020",30+ days ago,2022-05-23,2+ years’ experience in leading clinical opera...,3.4,\n\n\nCombine two of the fastest-growing field...,...,0,1,problem ability apply process skill knowledge...,1,payer cost saving staff training participate ...,interested required qualification undergradua...,1,1,UT,1
216,216,Health Information Manager,https://www.indeed.com/rc/clk?jk=3a9e8a5c9f2ba...,Redlands Community Hospital - MEDICAL RECORDS/HIM,"Redlands, CA 92373",30+ days ago,2022-05-23,"Health informatics, or related healthcare fiel...",NaN,\n Health Information Management Manager \n \n...,...,0,0,,0,,ahima rhit rhia baccalaureate degree associat...,0,0,CA,1
238,238,"Coding Specialist, Certified - Health Informat...",https://www.indeed.com/rc/clk?jk=823cabc91d0e3...,Hawaii Pacific Health,"Honolulu, HI",30+ days ago,2022-05-23,With the widespread computerization of health ...,4.1,\n\n\nHawai'i Pacific Health is a not-for-prof...,...,0,0,,1,,experience preferred qualification associate ...,0,0,HI,1
373,373,Remote Lead,https://www.indeed.com/rc/clk?jk=8cda48609d3cd...,Northwestern Medicine,"Remote in Chicago, IL",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,"\n\n At Northwestern Medicine, every patient ...",...,0,1,,0,issue reported end user training document tip...,meditech clinical preferred bachelor degree a...,1,1,IL,1
394,394,Remote Coding Specialist II,https://www.indeed.com/rc/clk?jk=3126d25924eb5...,Northwestern Medicine,"Remote in Chicago, IL 60611+3 locations",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,"\n\nAt Northwestern Medicine, every patient in...",...,0,0,,1,reflects patient service position train physi...,test preferred bachelor associate degree heal...,1,0,IL,1
510,510,Faculty Part-time - Health Informatics and Man...,https://www.indeed.com/rc/clk?jk=f8904c10a5184...,University of Massachusetts Lowell,"Lowell, MA 01854 (Pawtucketville area)",30+ days ago,2022-05-23,Demonstrated teaching experience in health inf...,4.3,\n\n\nThe Department of Public Health at the U...,...,1,1,oral written english communication skill addi...,0,,qualification required minimum master degree ...,0,1,MA,0
568,568,Remote Operations Coordinator,https://www.indeed.com/rc/clk?jk=531e4b08ae2c1...,Northwestern Medicine,"Remote in Chicago, IL 60611+3 locations",30+ days ago,2022-05-23,"At Northwestern Medicine, we pride ourselves o...",3.9,"\n\nAt Northwestern Medicine, every patient in...",...,0,0,,1,reflects patient service position train physi...,test preferred bachelor associate degree heal...,1,0,IL,1
586,586,"Manager, Health Informatics & Information Mana...",https://www.indeed.com/rc/clk?jk=2d84d03b966f5...,Community Health Systems,"Franklin, TN",3 days ago,2022-05-23,Facilitates electronic health record (EHR) sys...,3.2,Summary:\nThe HIIM Regulatory Manager is respo...,...,0,0,management posse critical thinking skill demo...,1,coordinate provide roi education training int...,qualification required education associate de...,0,0,TN,1
609,609,Instructor Health Information Technology,https://www.indeed.com/rc/clk?jk=99d85f6a187cb...,Milwaukee Area Technical College,Wisconsin+1 location,30+ days ago,2022-05-23,Hours vary based on departmental needs.,4.1,\n\n\n\n\nCandidates are required to submit co...,...,1,0,,0,institutional meeting promote education train...,required submit copy transcript degree obtain...,0,1,WI,1
676,676,Faculty FT - Health Information Management,https://www.indeed.com/rc/clk?jk=25f41937ef232...,Prince George's Community College,"Hybrid remote in Largo, MD 20774",30+ days ago,2022-05-23,Extent of pertinent work experience in 

In [113]:
# checking in how many jobs certain certifications was listed
mask6 = df_updated['cleanDesc'].str.contains(r'ecfmg', na=True)
df_updated[mask6]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,...,bachelor,graduate,skills,otherDegree,trainings,degrees,remote,college,states,certificationFlag
154,154,Physician - Deputy Chief Health Informatics Of...,https://www.indeed.com/rc/clk?jk=c031f95901348...,US Veterans Health Administration,"Richmond, VA 23249 (McGuire Veterans Hospital ...",30+ days ago,2022-05-23,Evaluate clinical computing systems; analyze a...,3.9,\n\n\nDuties\nThe Hunter Holmes McGuire VA Med...,...,0,1,sustaining spinal cord injury skilled nursing...,1,announcement applicant pending completion tra...,citizen accordance va policy degree doctor me...,0,1,VA,1
295,295,Physician (Chief Health Informatics Officer),https://www.indeed.com/rc/clk?jk=648add20f1e70...,US Veterans Health Administration,"Leavenworth, KS+2 locations",30+ days ago,2022-05-23,"The health care environment, including how bus...",3.9,\n\n\nDuties\nThe aim of clinical informatics ...,...,0,1,build critical competency knowledge skill pro...,0,database structure reporting service training...,citizen accordance va policy degree doctor me...,0,1,KS,1


In [47]:
# Top university degrees listed 
getHighDemandJobDescription(df_updated,'degrees')

        degree       bachelor         master     experience       required         health        related           year      preferred          field      education  qualification    informatics        science        nursing       computer     healthcare       clinical         public      associate     equivalent    information        minimum       business    requirement administration     accredited        college     management        program       relevant       doctoral     discipline           care        medical  certification    engineering           high       advanced      doctorate          skill       graduate     technology           lieu           work     university      statistic       position       medicine         higher            phd      candidate         school      technical       training         option          level   epidemiology     considered           data      knowledge    combination   professional       research    substituted           area        abi

In [48]:
# Top data visualization tools listed
getHighDemandJobDescription(df_updated,'visualization')

visualization          data          tool    experience      analysis       tableau     reporting        report     analytics      software       support      business    management         skill        design     dashboard  intelligence           sql        python   information     knowledge       perform         build     technique        health   statistical      database       insight     excellent      clinical        source    analytical       produce   requirement  presentation        create       variety       linkage          open       written      analytic       package       working       product    healthcare      develops     including    developing     effective      internal       project        result   communicate       process            sa            bi   application          work      customer   performance      research          shda       include   operational      resource        coding       testing         query      advanced     developer      required    pre

In [42]:
# Tool listed tools

#mask6 = df_updated['cleanDesc'].str.contains(r'js', na=True) & df_updated['cleanDesc'].str.contains(r'certificat', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'critical thinking', na=True) & df_updated['graduate']==1
#mask6 = df_updated['cleanDesc'].str.contains(r'java', na=True)
#mask6 = df_updated['Job_description'].str.contains(r' R ', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'power bi', na=True)
df_updated[mask6]
# Python 74
# Tableau 73
# Java 29
# R 24
# SPSS 14
# Power BI 13


,Unnamed: 0,Unnamed: 0.1,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,...,otherDegree,trainings,degrees,remote,college,states,certificationFlag,license,programming,visualization
13,13,13,"Health Informatics Expert, Remote",https://www.indeed.com/rc/clk?jk=060b177d5794f...,Siemens Healthineers,"Remote in Malvern, PA 19355",30+ days ago,2022-05-23,Siemens Healthineers has recently announced th...,3.8,...,0,,skill success role bachelor degree data analy...,1,0,PA,0,,,
44,44,44,"Associate Manager, Health Informatics and Repo...",https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Monitor emerging topics and news in health pol...,NaN,...,0,procedure provide guidance mentoring training...,manager chia qualification bachelor degree so...,1,1,MA,0,,,excel powerpoint familiarity data visualizati...
79,79,79,CHESS: Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=ec9aca1a1cb77...,MedCost LLC,Remote,4 days ago,2022-05-23,Familiar with clinical and administrative work...,3.1,...,1,data solution department process training man...,position qualification required bachelor degr...,1,0,0,0,,,management software tool data visualization r...
92,92,92,POPULATION HEALTH ANALYST,https://www.indeed.com/rc/clk?jk=975cf3ecd0605...,Whitman-Walker Health,"Washington, DC 20009 (Logan Circle area)",13 days ago,2022-05-23,Synthesizes data from across a wide variety of...,4.6,...,1,data dashboard dashboard usage training drive...,education experience preferred bachelor degre...,0,0,WA,1,,,internal expert data analysis visualization s...
102,102,102,MHS Health Informatics - Senior Consultant,https://www.indeed.com/rc/clk?jk=333c280f648d1...,Guidehouse,"Tysons Corner, VA",5 days ago,2022-05-23,Provide health informatics subject matter expe...,3.3,...,0,,requirement qualification required bachelor d...,0,0,VA,1,,,ability develop dashboard data visualization ...
118,118,118,HEALTH INFORMATICS ANALYST.7036200,https://www.indeed.com/rc/clk?jk=5707e19b19614...,Dallas County,"Dallas, TX (Design District area)",30+ days ago,2022-05-23,"Analyzes, designs, implements and maintains he...",3.5,...,0,,education experience equivalent bachelor degr...,0,1,TX,0,posse valid texas driver license good driving...,,
232,232,232,Data Coordinator,"https://www.indeed.com/company/TCA-Health,-Inc...",TCA HEALTH INC,"Chicago, IL 60628 (Altgeld Gardens area)",4 days ago,2022-05-23,"Bachelor’s Degree in public health, health sci...",NaN,...,0,capacity creating maintain documentation trai...,metric education experience bachelor degree p...,0,1,IL,0,,,experience emr athena data visualization soft...
233,233,233,Junior Pricing Analyst,https://www.indeed.com/company/ScriptSave/jobs...,Scriptsave,Remote,10 days ago,2022-05-23,"BA/BA degree in public health, statistics, hea...",3.6,...,1,product documentation technical assistance tr...,education experience ba ba degree public heal...,1,0,0,0,,,
278,278,278,Informatics Data Analyst,https://www.indeed.com/rc/clk?jk=1f9a16d9f00f0...,Community Health Group,"Remote in Chula Vista, CA 91914",30+ days ago,2022-05-23,Community Health Group is a locally based non-...,3.7,...,1,documentation specification flowchart procedu...,image qualification education bachelor degree...,1,0,CA,0,,,
297,297,297,Healthcare Data Analyst,https://www.indeed.com/company/Seven-Hills-Fou...,Seven Hills Foundation,"Worcester, MA 01603",2 days ago,2022-05-23,"1. Bachelor’s degree in business, health infor...",3.4,...,1,,mccn coordinator qualification bachelor degre...,0,0,MA,1,collaborative relationship valid driver licen...,,


In [48]:
# Jobs that requests bachelors degrees
df_updated[df_updated['bachelor']==1]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,Salary,JobType,JobShift,cleanDesc,certifications,skilss,bachelor,graduate,skills,otherDegree
32,32,Health Informatics and Reporting Analyst,https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Working knowledge of the Massachusetts health ...,NaN,\nThe mission of the Center for Health Informa...,NaN,NaN,NaN,mission center health information analysis chi...,,addition analyst strong analytic skill order ...,1,1,addition analyst strong analytic skill order ...,0
44,44,"Associate Manager, Health Informatics and Repo...",https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Monitor emerging topics and news in health pol...,NaN,"\nAn independent state agency, the Center for ...",NaN,NaN,NaN,independent state agency center health informa...,,question strong data management skill includi...,1,0,question strong data management skill includi...,0
49,49,Adjunct Faculty – Healthcare Informatics,https://www.indeed.com/rc/clk?jk=e8894c65cd061...,MCPHS University,Remote,30+ days ago,2022-05-23,Minimum of a master’s degree in health science...,3.9,The School of Arts and Sciences seeks an adjun...,NaN,NaN,NaN,school art science seek adjunct faculty member...,,environment excellent organizational communic...,1,1,environment excellent organizational communic...,0
50,50,Health Informatics Manager,https://www.indeed.com/company/Orange-County-G...,Orange County Government,"Hillsborough, NC 27278",30+ days ago,2022-05-23,The work involves the responsibility for promo...,NaN,\nThe Orange County Health Department has an o...,NaN,NaN,NaN,orange county health department opening health...,staff year preferred license certification dr...,,1,0,,0
58,58,"IM Field Medical, Outcomes & Analytics (non-MD)",https://www.indeed.com/rc/clk?jk=40c7be08c49b2...,Pfizer,Utah,11 days ago,2022-05-23,"Experience analyzing health care data (claims,...",4.2,"\n\n\n IM Field Medical, Outcomes & Analytic...",NaN,NaN,NaN,field medical outcome analytics md role summar...,,implement demonstrate excellent judgment skil...,1,0,implement demonstrate excellent judgment skil...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,799,Health and Safety Fall Co-Op Positions,https://www.indeed.com/rc/clk?jk=e296525257a4e...,Cummins Inc.,"Columbus, IN",30+ days ago,2022-05-23,Fundamentals of Health and Safety - Comprehens...,3.8,\n\nHealth and Safety Fall Co-Op Positions\nDe...,NaN,NaN,NaN,health safety fall op position description hea...,,compliance legislation risk management skill ...,1,1,compliance legislation risk management skill ...,0
801,801,SYSTEMS PROJECT CONSULTANT,https://www.indeed.com/rc/clk?jk=1cb57a27f8493...,The State of Florida,"Tallahassee, FL 32308",30+ days ago,2022-05-23,The selected candidate will assist in detectin...,3.5,\n\nRequisition No: 582384\nAgency: Agency for...,NaN,NaN,NaN,requisition agency agency health care administ...,preferred college education licensure certifi...,mpi index shtml knowledge skill ability excel...,1,1,mpi index shtml knowledge skill ability excel...,0
805,805,Data Analyst,https://www.indeed.com/company/NYSTEC/jobs/Dat...,NYSTEC,"Albany, NY 12207+1 location",20 days ago,2022-05-23,Use of health informatics or Medicaid/Medicare...,4.0,\nAbout Us: NYSTEC is a nonprofit technology c...,NaN,NaN,NaN,nystec nonprofit technology consulting company...,,critical thinking problem solving skill key r...,1,0,critical thinking problem solving skill key r...,0
811,811,Clinical Informatics Specialist 3 - * Days - 4...,https://www.indeed.com/company/University-of-M...,Metro Health - University of Michigan Health,"Wyoming, MI",Hiring ongoing,2022-05-23,Certification in nursing informatics preferred.,3.7,\nClinical Inf

In [49]:
# Jobs that requests bachelors degrees
df_updated[df_updated['graduate']==1]

,Unnamed: 0,Job_titles,Job_URLS,Company,location,PostDate,ExtractDate,Summary,Job_ratings,Job_description,Salary,JobType,JobShift,cleanDesc,certifications,skilss,bachelor,graduate,skills,otherDegree
32,32,Health Informatics and Reporting Analyst,https://www.indeed.com/company/The-Center-for-...,Center For Health Information And Analysis,"Hybrid remote in Boston, MA 02116",12 days ago,2022-05-23,Working knowledge of the Massachusetts health ...,NaN,\nThe mission of the Center for Health Informa...,NaN,NaN,NaN,mission center health information analysis chi...,,addition analyst strong analytic skill order ...,1,1,addition analyst strong analytic skill order ...,0
37,37,Informatics Analyst,https://www.indeed.com/rc/clk?jk=d4b54cd7730b1...,The University of Chicago Medicine,"Chicago, IL 60637 (Hyde Park area)",3 days ago,2022-05-23,Knowledge of the US health care system preferred.,3.8,\n\n Job Description: \n Join UChicago Medi...,NaN,NaN,NaN,job description join uchicago medicine informa...,,care preferred strong analytical skill excel ...,0,1,care preferred strong analytical skill excel ...,0
39,39,Clinical Informatics Specialist,https://www.indeed.com/rc/clk?jk=455c2d5b2baf1...,University of California San Francisco,Remote,17 days ago,2022-05-23,Broadly encompassing or in-depth knowledge of ...,4.1,\n\n\n\n\n\n The California Department of ...,NaN,NaN,NaN,california department public health covid data...,,working public health strong skill statistica...,0,1,working public health strong skill statistica...,0
49,49,Adjunct Faculty – Healthcare Informatics,https://www.indeed.com/rc/clk?jk=e8894c65cd061...,MCPHS University,Remote,30+ days ago,2022-05-23,Minimum of a master’s degree in health science...,3.9,The School of Arts and Sciences seeks an adjun...,NaN,NaN,NaN,school art science seek adjunct faculty member...,,environment excellent organizational communic...,1,1,environment excellent organizational communic...,0
53,53,Health Informatics Part Time Lecturer,https://www.indeed.com/rc/clk?jk=2328b797f7e07...,Northeastern University,"Boston, MA+1 location",30+ days ago,2022-05-23,Teach and develop on-line or on-campus courses...,4.2,\n\n\n About the Opportunity\n \n\n\n Te...,NaN,NaN,NaN,opportunity teach develop line campus health i...,,,0,1,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,788,Clinical Informatics Analyst,https://www.indeed.com/rc/clk?jk=3a02f7e4353ba...,Central Peninsula Hospital,"Soldotna, AK 99669",26 days ago,2022-05-23,Advanced Degree in Healthcare Informatics rela...,3.7,\n\n\n\n\n\nPOSITION SUMMARY: The primary func...,NaN,NaN,NaN,position summary primary function position pro...,skill hl edi proficiency certified profession...,high motivation excellent communication skill...,0,1,high motivation excellent communication skill...,0
799,799,Health and Safety Fall Co-Op Positions,https://www.indeed.com/rc/clk?jk=e296525257a4e...,Cummins Inc.,"Columbus, IN",30+ days ago,2022-05-23,Fundamentals of Health and Safety - Comprehens...,3.8,\n\nHealth and Safety Fall Co-Op Positions\nDe...,NaN,NaN,NaN,health safety fall op position description hea...,,compliance legislation risk management skill ...,1,1,compliance legislation risk management skill ...,0
801,801,SYSTEMS PROJECT CONSULTANT,https://www.indeed.com/rc/clk?jk=1cb57a27f8493...,The State of Florida,"Tallahassee, FL 32308",30+ days ago,2022-05-23,The selected candidate will assist in detectin...,3.5,\n\nRequisition No: 582384\nAgency: Agency for...,NaN,NaN,NaN,requisition agency agency health care administ...,preferred college education licensure certifi...,mpi index shtml knowledge skill ability excel...,1,1,mpi index shtml knowledge skill ability excel...,0
814,814,Public Health Informaticist,https://www.indeed.com/rc/clk?jk=be30904d64b13...,Weber County,"Ogden, UT 84401",30+ days ago,2022-05-23,Ability to integrate and incorporate informati...,2.7,\n\nOVERVIEW:\n This is a grant funded positio...,NaN,NaN,NaN,overvie

In [109]:
# The average indeed ratings of Institutions that listed the jobs
print(df_updated.agg({"Job_ratings": ["mean"]}))
print(len(df_updated.Company.unique()))

      Job_ratings
mean     3.730213
592


In [103]:
# Save the results of analysis in csv
df_updated.to_csv('JobDetailsAnalysis.csv')